# Homework - Grover MaxCut

The places where you have enter code are marked with `# YOUR CODE HERE`.

In [1]:
import cirq
from cirq import H, X, Y, Z, CX, inverse

## Question 1 (4 points)

Write a function, `oracle010`, that implements an oracle that marks the state $|010 \rangle$. The function `oracle010` has

* input: `qq`, a 3-qubit register 
* returns: None

The function should append a sequence of gates to `qq` to mark the state $|010\rangle$ only. Don't append any measurements to `qq`.

To help you test the function, we have provided the `grover_diffusion` and `grover` functions.

In [2]:
def oracle010(qq):
    # YOUR CODE HERE
    circuit = cirq.Circuit()
    yield X.on_each(qq[0:3:2])
    yield Z(qq[1]).controlled_by(qq[0], qq[2])
    yield X.on_each(qq[0:3:2])

In [3]:
# visualize your implemented gates
qqTest = cirq.LineQubit.range(3)
circuit = cirq.Circuit()
circuit.append(oracle010(qqTest))
circuit

0: ───X───@───X───
          │
1: ───────@───────
          │
2: ───X───@───X───

In [4]:
# To check your solution, we need some to implement grover
def grover_diffusion(qq,n):
    yield H.on_each(*qq)
    yield X.on_each(*qq)
    yield Z(qq[n-1]).controlled_by(*(qq[0:n-1]))
    yield X.on_each(*qq)
    yield H.on_each(*qq)

In [5]:
def grover(trials_number):
    n=3
    qq = cirq.LineQubit.range(n)
    circuit = cirq.Circuit()
    circuit.append(H.on_each(*qq))  

    for i in range(2):
        circuit.append(oracle010(qq))
        circuit.append(grover_diffusion(qq,n))
    circuit.append(cirq.measure(*qq, key='result'))

    # determine the statistics of the measurements
    s = cirq.Simulator() 
    
    samples = s.run(circuit, repetitions=trials_number)

    def bitstring(bits):
        return "".join(str(int(b)) for b in bits)

    counts = samples.histogram(key="result",fold_func=bitstring)
    print(counts)
    return counts.get('010')

In [6]:
# run grover to test if your function gives the right answer
grover(100)

Counter({'010': 95, '111': 3, '001': 1, '101': 1})


95

In [7]:
# hidden tests in this cell will be used for grading.

## Question 2 (6 points)

Write an oracle for the graph described below to check whether it admits a valid 2-coloring. 

The function `oracle2` has

* input: `qq`, a 12-qubit register 
* returns: None

The function should append a sequence of gates to `qq` to check for a valid coloring. Don't append any measurements to `qq`.

You can test the oracle with the provided `grover_diffusion`, `grover` and `oracle_computation2` functions.



$G$ has 5 vertices. Use qubits 0-4 for the vertices, 5-10 for the edges and 11 as the ancilla.

The list of edges:
(0,3)
(0,4)
(1,3)
(1,4)
(2,3)
(2,4)


In [8]:
def oracle2(qq):
    # YOUR CODE HERE
    # we will store on 5th qubit and check on edges (0,3)
    yield CX(qq[0], qq[5])
    yield CX(qq[3], qq[5])
    # now storing it at 6th qubit and check on edges (0,4)
    yield CX(qq[0], qq[6])
    yield CX(qq[4], qq[6])
    # now storing at 7th qubit and checking on edges (1,3)
    yield CX(qq[1], qq[7])
    yield CX(qq[3], qq[7])
    # now storing at 8th qubit and checking on edges (1,4)
    yield CX(qq[1], qq[8])
    yield CX(qq[4], qq[8])
    # now storing at 9th qubit and checking on edges (2,3)
    yield CX(qq[2], qq[9])
    yield CX(qq[3], qq[9])
    # now storing at 10th qubit and checking on edges (2,4)
    yield CX(qq[2], qq[10])
    yield CX(qq[4], qq[10])
    
    yield X(qq[11]).controlled_by(*qq[5:10])

In [9]:
# We need some code so you can check your solution
def oracle_computation2(qq):
    yield oracle2(qq)
    yield Z(qq[11])
    yield inverse(oracle2(qq))  

In [10]:
def grover2(trials_number):    
    import cirq
    from cirq import X, H, Z, inverse, CX
    s = cirq.Simulator()

    qq = cirq.LineQubit.range(12)
    n=5
    
    circuit = cirq.Circuit()
    circuit.append(H.on_each(*(qq[0:n])))
    for i in range(2):
        circuit.append(oracle_computation2(qq))
        circuit.append(grover_diffusion(qq,n))

    circuit.append(cirq.measure(*(qq[0:n]), key='result'))

    # determine the statistics of the measurements
    samples = s.run(circuit, repetitions=trials_number)
    result = samples.measurements["result"]

    def bitstring(bits):
        return "".join(str(int(b)) for b in bits)

    counts = samples.histogram(key="result",fold_func=bitstring)
    return counts

In [11]:
#You can use this cell to test your solution
shots=1000
grover2(shots)

Counter({'11100': 463,
         '00011': 425,
         '00110': 3,
         '10100': 7,
         '01010': 4,
         '11001': 4,
         '00000': 8,
         '11000': 4,
         '10101': 4,
         '00001': 4,
         '10110': 5,
         '00010': 3,
         '10011': 4,
         '00111': 6,
         '01110': 11,
         '10010': 4,
         '00101': 2,
         '11101': 3,
         '01000': 9,
         '01001': 2,
         '11010': 2,
         '01011': 5,
         '01111': 4,
         '00100': 3,
         '01101': 3,
         '10111': 2,
         '10001': 1,
         '10000': 4,
         '11011': 1})

In [12]:
# hidden tests in this cell will be used for grading.

## Question 3 (10 points)

Write an oracle for the graph described below to check whether there exists a coloring with 4 edges connecting vertices with different colors.

The function `oracle3` has

* input: `qq`, a 13-qubit register 
* returns: None

The function should append a sequence of gates to `qq` to check whether there exists a coloring with 4 edges connecting vertices with different colors. Don't append any measurements to `qq`.

You can test the oracle with the provided `grover_diffusion`, `grover` and `oracle_computation3` functions.


$G$ has 4 vertices. Use qubits 0-3 for the vertices, 4-8 for the edges, 9-11 for the addition and 12 as the ancilla.

The list of edges: 
(0,1)
(0,2)
(0,3)
(1,2)
(1,3)

In [13]:
def oracle3(qq):
    # YOUR CODE HERE
    
    # we will store on 4th qubit and check on edges (0,1)
    yield CX(qq[0],qq[4])
    yield CX(qq[1],qq[4])
    
    # now storing it at 5th qubit and check on edges (0,2)
    yield CX(qq[0],qq[5])
    yield CX(qq[2],qq[5])
    
    # now storing at 6th qubit and checking on edges (0,3)
    yield CX(qq[0],qq[6])
    yield CX(qq[3],qq[6])
    
    # now storing at 7th qubit and checking on edges (1,2)
    yield CX(qq[1],qq[7])
    yield CX(qq[2],qq[7])
    
    # now storing at 8th qubit and checking on edges (1,3)
    yield CX(qq[1],qq[8])
    yield CX(qq[3],qq[8])
    
    #adding values of edge (0,1)
    yield CX(qq[4],qq[9])
    
    #adding value of edge (0,2)
    yield CCX(qq[5],qq[9],qq[10])
    yield CX(qq[5],qq[9])
    
    #adding value of edge(0,3)
    yield CCX(qq[6],qq[9],qq[10])
    yield CX(qq[6],qq[9])
    
    #adding value of edge(1,2)
    yield X(qq[11]).controlled_by(qq[7], qq[9],qq[10])
    yield CCX(qq[7], qq[9], qq[10])
    yield CX(qq[7], qq[9])
    
    #adding value of edge(1,3)
    yield X(qq[11]).controlled_by(qq[8], qq[9],qq[10])
    yield CCX(qq[8], qq[9], qq[10])
    yield CX(qq[8], qq[9])
    
    # checking whether qubits qq[11], q[10], q[9] have 100 or not
    yield X(qq[10])
    yield X(qq[9])
    
    # We verify by applying X controlled by [9:11] qubits on ancilla qq[12],see if qq[11], qq[10], q[9] is 111, qq[12] =1 
    yield X(qq[12]).controlled_by(qq[9],qq[10],qq[11])


In [14]:
# We need some code so you can check your solution
def oracle_computation3(qq):
    yield oracle3(qq)
    yield Z(qq[12])
    yield inverse(oracle3(qq))  

In [15]:
import cirq
from cirq import X, H, Z, inverse, CX, CCX
    
def grover3(trials_number):    
    s = cirq.Simulator()

    qq = cirq.LineQubit.range(13)
    n=4

    circuit = cirq.Circuit()
    circuit.append(H.on_each(*(qq[0:n])))
    for i in range(2):
        circuit.append(oracle_computation3(qq))
        circuit.append(grover_diffusion(qq,n))

    circuit.append(cirq.measure(*(qq[0:n]), key='result'))

    # determine the statistics of the measurements
    samples = s.run(circuit, repetitions=trials_number)
    result = samples.measurements["result"]

    def bitstring(bits):
        return "".join(str(int(b)) for b in bits)

    counts = samples.histogram(key="result",fold_func=bitstring)
    return counts

In [16]:
#You can use this cell to test your solution
shots=1000
grover3(shots)

Counter({'0011': 474,
         '1110': 5,
         '1100': 471,
         '1000': 4,
         '0110': 4,
         '0001': 4,
         '0010': 4,
         '0000': 4,
         '1101': 7,
         '0100': 4,
         '1111': 5,
         '0101': 2,
         '1001': 5,
         '1011': 3,
         '0111': 4})

In [17]:
# hidden tests in this cell will be used for grading.